Tobias Kuhlmann, Karlsruhe Institute of Technology (KIT), tobias.kuhlmann@student.kit.edu

In [1]:
%matplotlib inline

# Pretty Display of Variables
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Double resolution plotting for retina display
%config InlineBackend.figure_format ='retina'

import numpy as np
import pylab as pl
import pandas as pd
import matplotlib.pyplot as plt

import glob
import os

import datetime as dt




## Data

##### Data needed
- Return time series to calculate rolling window stock-stock correlations $\rho^{P}_{ij,t}$
- Stock weights in index over time 
- Implied volatilities of every single stock i $\sigma^{Q}_{i,t}$
- Implied volatilities of Index $\sigma^{Q}_{M,t}$
- Use stock-stock correlations, implied volatilities of single stocks and index implied volatility to calculate $\alpha_t$


In [2]:
# Validate relative file path and list files
os.listdir("../Option_Implied_Beta_Tobias/")

['Single_Stock_Skewness',
 'Fundamentals_SP500_Full.xlsx',
 'usdOIScurve.csv',
 '.DS_Store',
 '0_Paper',
 'Calculation_Process2.ipynb',
 '2019_04_26_Cpy_Python_Code-Skewness_v8.ipynb',
 'instrumentid_and_symbol.csv',
 'SP500',
 'CRAMnoarbEOD_USOPT0007588D1_measuresByMaturity.csv',
 'Fundamentals_SP500_Full copy.xlsx',
 '.ipynb_checkpoints',
 '~$Fundamentals_SP500_Full.xlsx']

###### Import instrument id on ticker mapping

In [3]:
id_ticker_map = pd.read_csv("../Option_Implied_Beta_Tobias/instrumentid_and_symbol.csv")
id_ticker_map.shape
id_ticker_map.head(1)


(7590, 3)

,instrumentid,symbol,name
0,USOPT0000001D1,1R,NFX (OPIS) Mont Belvieu Non-LST Propane Future


##### Implied volatilities of Index $\sigma^{Q}_{M,t}$
- Question: Daiana file: Bereits bereinigt? Maturities 30 days out, interpolated out
- Ein Tag: 14 Tage, 41 Tage -> linear interpolieren auf 30 Tage
- Interpolation aus Simon's vol surface paper


In [ ]:
sp500_options = pd.read_csv("../Option_Implied_Beta_Tobias/CRAMnoarbEOD_USOPT0007588D1_measuresByMaturity.csv", sep=";")
sp500_options.shape
sp500_options.head(1)



##### Implied volatilities of every single stock i $\sigma^{Q}_{i,t}$
Read in risk neutral measures for single stocks and combine in dataframe

##### Data structures and implementation
1. Create pandas data structure with dates as index
2. Filter and interpolate for every csvfile=instrument
3. append rowwise with instruments as columns and dates, maturities as index
4. move back to long fomat

In [ ]:
# the path to your csv file directory
mycsvdir = '../Option_Implied_Beta_Tobias/Single_Stock_Skewness/'

# get all the csv files in that directory
csvfiles_w_path = glob.glob(os.path.join(mycsvdir, '*.csv'))

# loop through the files and read them in with pandas
stock_risk_neutral_measures = pd.DataFrame(columns=['id', 
                                     'loctimestamp',
                                     'daystomaturity', 
                                     'bakshiVariance',
                                     'SVIX']) 
for csvfile in csvfiles_w_path[0:1]:
    df = pd.read_csv(csvfile, usecols=['loctimestamp',
                                     'daystomaturity', 
                                     'bakshiVariance',
                                     'interpolmaturity'])
    # add column with instrument id
    filename_wo_ext=os.path.basename(csvfile)
    df['id'] = os.path.splitext(filename_wo_ext)[0].partition("_")[0]
    # drop interpolated maturities, we interpolate ourselves
    df = df[df['interpolmaturity']==0].drop(columns='interpolmaturity')
    # interpolate following Walther, Ulrich(2019)
    
    # append
    stock_risk_neutral_measures = stock_risk_neutral_measures.append(df[['id',
                                     'loctimestamp',
                                     'daystomaturity', 
                                     'bakshiVariance',]])

print(f"shape: {stock_risk_neutral_measures.shape}")
print(f"Unique instrument id: {stock_risk_neutral_measures.id.unique().shape}")
stock_risk_neutral_measures.head(5)